<a href="https://colab.research.google.com/github/lsGee/TIL/blob/master/01.Python/python_ds_4_randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Load

## train data

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/bike/train.csv")

print(train.shape)

(10886, 12)


In [ ]:
train["datetime"] = pd.to_datetime(train["datetime"])

train["datetime-year"] = train["datetime"].dt.year
train["datetime-month"] = train["datetime"].dt.month
train["datetime-day"] = train["datetime"].dt.day
train["datetime-hour"] = train["datetime"].dt.hour
train["datetime-minute"] = train["datetime"].dt.minute
train["datetime-second"] = train["datetime"].dt.second

print(train.shape)
train.head()

(10886, 18)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute,datetime-second
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0,0,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1,0,0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2,0,0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3,0,0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4,0,0


## test data

In [ ]:
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/bike/test.csv")

print(test.shape)

(6493, 9)


In [ ]:
test["datetime"] = pd.to_datetime(test["datetime"])

test["datetime-year"] = test["datetime"].dt.year
test["datetime-month"] = test["datetime"].dt.month
test["datetime-day"] = test["datetime"].dt.day
test["datetime-hour"] = test["datetime"].dt.hour
test["datetime-minute"] = test["datetime"].dt.minute
test["datetime-second"] = test["datetime"].dt.second

print(test.shape)

test.head()

(6493, 15)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute,datetime-second
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,0,0,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,1,0,0
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,2,0,0
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,0,0
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,4,0,0


# Random Forest

## Train

In [ ]:
feature_names = ["season", "holiday", "workingday", "weather", 
                 "temp", "atemp", "humidity", "windspeed",
                 "datetime-year", "datetime-month", "datetime-day", 
                 "datetime-hour", "datetime-minute", "datetime-second"]

In [ ]:
x_train = train[feature_names]

# train과 test 서로 바꿔쓰는 실수 있으니 항상 확인해주기!
print(x_train.shape)  
x_train.head()

(10886, 14)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute,datetime-second
0,1,0,0,1,9.84,14.395,81,0.0,2011,1,1,0,0,0
1,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,1,0,0
2,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,2,0,0
3,1,0,0,1,9.84,14.395,75,0.0,2011,1,1,3,0,0
4,1,0,0,1,9.84,14.395,75,0.0,2011,1,1,4,0,0


In [ ]:
x_test = test[feature_names]

# train과 test 서로 바꿔쓰는 실수 있으니 항상 확인해주기!
print(x_test.shape)
x_test.head()

(6493, 14)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute,datetime-second
0,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,0,0,0
1,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,1,0,0
2,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,2,0,0
3,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,0,0
4,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,4,0,0


In [ ]:
label_name = "count"
y_train = train[label_name]

print(y_train.shape)
y_train.head()

(10886,)


0    16
1    40
2    32
3    13
4     1
Name: count, dtype: int64

[RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html?highlight=random%20forest%20regressor#sklearn.ensemble.RandomForestRegressor)의 파라미터를 조절해 성능을 더욱 향상시킬 수 있다.

**[ Hyper Parameter]**
* `n_estimators` :  트리의 개수
* `max_depth` :  트리의 최대 깊이
* `max_features` :  전체 features 중 사용할 최대 개수(%) 예) 0.5(50%)

등등

**참고) 랜덤포레스트 이외의 모델**
* Gradient Boosting
* XGBoost
* LightGBM

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=10,
                              n_jobs=-1,
                              random_state=37)
model

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=-1, oob_score=False,
                      random_state=37, verbose=0, warm_start=False)

## Evaluate (Cross validation)

**`cross_val_predict()` 파라미터 설명**
* `cv` : 조각 개수

**참고)** Train과 Fit 사이에 넣어주기! (나중에 한번에 모두 실행시킬 때, 순서 중요)

In [ ]:
# Model Validation
from sklearn.model_selection import cross_val_predict 

# kaggle은 cv 가 20일 때 가장 정확하대..!
y_predict = cross_val_predict(model, x_train, y_train, cv = 20)

print(y_predict.shape)
y_predict

(10886,)


array([ 33.9,  27.1,  21.5, ..., 205. , 186.7, 136.4])

In [ ]:
# Evaluation Metric (1-1)직접 계산하기(MAE)
score = abs(y_train - y_predict).mean()
score

36.46493661583686

In [ ]:
# Evaluation Metric (1-2)scikit-learn 에서 MAE 구현하기
from sklearn.metrics import mean_absolute_error

score = mean_absolute_error(y_train, y_predict)
print("MAE:", score)

MAE: 36.46493661583686


In [ ]:
#Evaluation Metric (2) MSE & RMSE
# scikit-learn에서는 지원하지 않고 있음
# 따라서 MSE에서 루트를 씌워줘서 계산(numpy 사용)
from sklearn.metrics import mean_squared_error   # MSE
import numpy as np

score = mean_squared_error(y_train, y_predict)
print("MSE:", score)
print("RMSE:", np.sqrt(score))

MSE: 3251.063039683998
RMSE: 57.01809396747666


In [ ]:
# Evaluation Metric (3)RMSLE
from sklearn.metrics import mean_squared_log_error

score = mean_squared_log_error(y_train, y_predict)
score = np.sqrt(score)
print(f"RMSLE: {score:.5f}")

RMSLE: 0.41209


## Fit

**참고) `random_state`**

* 데이터 분할 시 셔플이 이뤄져서 예측결과 소숫점이 매번 달라지는데, 이를 고정하기 위한 값

* 아무 값이나 넣어줘도 OK!

In [ ]:
model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=-1, oob_score=False,
                      random_state=37, verbose=0, warm_start=False)

## Predict

In [ ]:
prediction_list = model.predict(x_test)

print(prediction_list.shape)
prediction_list

(6493,)


array([ 13.8,   7.1,   4.7, ..., 161.5, 116. ,  69.3])